In [1]:
!pip install ultralytics
!pip install -U ipywidgets

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/5c/1c/8e6e50fea7b6e89a953af59ced80d6c6c1a140fa214a39a019d3cb807290/ultralytics-8.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 326.9 kB/s eta 0:00:00
  Obtaining dependency information for hub-sdk>=0.0.2 from https://files.pythonhosted.org/packages/2f/e0/b3db53becb3c0fa6ad9ddf4d047f4b7dbe85987594bcdd4d1b247f951522/hub_sdk-0.0.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.8/699.8 kB 1.3 MB/s eta 0:00:00
  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.9 from https://files.pythonhosted.org/packages/29/03/107d96077c4befed191f7ad1a12c7b52a8f9d2778a5836d59f9855c105f6/widgetsnbextension-4.0.9-py3-none-any.whl.me

In [2]:
!mkdir -p /kaggle/temp/train/labels
!mkdir -p /kaggle/temp/train/images

!mkdir -p /kaggle/temp/val/labels
!mkdir -p /kaggle/temp/val/images

In [3]:
!cp /kaggle/input/kidney-1-dense-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-1-dense-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-1-voi-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-1-voi-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-2-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-2-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-3-sparse-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-3-sparse-to-yolo/train/images/* /kaggle/temp/train/images

In [4]:
!cp /kaggle/input/kidney-3-dense-to-yolo/train/labels/* /kaggle/temp/val/labels
!cp /kaggle/input/kidney-3-dense-to-yolo/train/images/* /kaggle/temp/val/images

In [5]:
# !tail /kaggle/input/kidney-1-voi-to-yolo/train/kidney_1_voi.json

# Write yolo dataset yaml

In [6]:
import yaml

In [7]:
!mkdir train

In [8]:
train_path = '/kaggle/temp/train'
val_path = '/kaggle/temp/val'
names = {0: 'Vessel'}
yaml_path = 'train/train.yaml'

# Number of classes
nc = len(names)

# Create a dictionary with the required content
yaml_data = {
    'names': names,
    'nc': nc,
    'test': '',
    'train': train_path,
    'val': val_path
}

# Write the dictionary to a YAML file
with open(yaml_path, 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False)

In [9]:
!cat {yaml_path}

names:
  0: Vessel
nc: 1
test: ''
train: /kaggle/temp/train
val: /kaggle/temp/val


In [10]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb-api-key")
wandb.login(key=wandb_api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')

100%|██████████| 6.73M/6.73M [00:00<00:00, 68.7MB/s]


In [12]:
project = 'hacking_human_vasculature'
name = 'kidney_3_dense_val_rest_train_img_size_1024'

results = model.train(data=yaml_path,
                      project=project,
                      name=name,
                      epochs=200,
                      time=5,
                      patience=30, #I am setting patience=0 to disable early stopping.
                      batch=32,
                      imgsz=1024,
                      device=[0, 1],
                      save_period=1,
                      cache=True
                     )

Ultralytics YOLOv8.1.1 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
                                                     CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=train/train.yaml, epochs=200, time=5, patience=30, batch=32, imgsz=1024, save=True, save_period=1, cache=True, device=[0, 1], workers=8, project=hacking_human_vasculature, name=kidney_3_dense_val_rest_train_img_size_1024, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=

100%|██████████| 755k/755k [00:00<00:00, 13.4MB/s]
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TensorBoard: Start with 'tensorboard --logdir hacking_human_vasculature/kidney_3_dense_val_rest_train_img_size_1024', view at http://localhost:6006/


...

      43/43      7.42G     0.6404       0.58     0.3681     0.7811        414       1024:  82%|████████▏ | 177/217 [04:07<00:55,  1.40s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:20<00:00,  1.61it/s]


                   all       1035      41540       0.51      0.344      0.288      0.181      0.374      0.268      0.217      0.107

43 epochs completed in 5.007 hours.
Optimizer stripped from hacking_human_vasculature/kidney_3_dense_val_rest_train_img_size_1024/weights/last.pt, 6.8MB
Optimizer stripped from hacking_human_vasculature/kidney_3_dense_val_rest_train_img_size_1024/weights/best.pt, 6.8MB

Validating hacking_human_vasculature/kidney_3_dense_val_rest_train_img_size_1024/weights/best.pt...
Ultralytics YOLOv8.1.1 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
                                                     CUDA:1 (Tesla T4, 15102MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [01:01<00:00,  1.87s/it]


                   all       1035      41540       0.51      0.344      0.294      0.185      0.374      0.266      0.219      0.108
Speed: 1.1ms preprocess, 7.0ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to hacking_human_vasculature/kidney_3_dense_val_rest_train_img_size_1024


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▆███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
wandb:                  lr/pg1 ▃▆███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
wandb:                  lr/pg2 ▃▆███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
wandb:        metrics/mAP50(B) ▄▃▁▄▅▅▅▆▅▆▇▆▆▇▆▆▆▇▇▇▇▇▆▇▇▇▇▇▇▇██████████
wandb:        metrics/mAP50(M) ▃▂▁▄▅▄▆▇▆▇█▆▇▇▇▇▆▇▇█▇▇▆█▇▇▇█▇▇██████████
wandb:     metrics/mAP50-95(B) ▃▂▁▃▃▄▄▅▄▅▆▅▅▆▅▅▅▆▇▆▆▅▆▆▆▆▇▇▇▇▇█████████
wandb:     metrics/mAP50-95(M) ▂▁▁▃▄▄▅▇▅��█▆▇▇▇▆▅▇▇▇▇▇▆▇▇▇▇█▇▇▇▇█▇▇▇██▇█
wandb:    metrics/precision(B) ▄▂▁▅▄▅▅▆▅▆▇▆▇▇▇▇▆▇▇▇▇▇▆▇▇▇██▇▇▇█████████
wandb:    metrics/precision(M) ▁▁▃▄▁▂▆▇▅▅█▅▆▆▆▄▅▆▆▆▆▆▅▆▆▅▆▇▅▆▇▇▆▆▆▇▇▇▇▇
wandb:       metrics/recall(B) ▅▃▁▄▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████
wandb:       metrics/recall(M) ▄▃▁▄▆▅▇█▇█▇██▇█▇▆▇▇█▇█▇▇██▇█▇▇▇▇█▇▇███▇▇
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch

In [13]:
wandb.finish()